<a href="https://colab.research.google.com/github/ikfaisal/dsia/blob/master/DSIA_Lab_2_1_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3: 
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [163]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [164]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [165]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

In [166]:
c.execute('SELECT * FROM houses').fetchall()

[]

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [167]:
# A
last_sale = (None, 4000, 5, 22, 619000)

c.execute('INSERT INTO houses VALUES(?,?,?,?,?)',last_sale)

# Execute Command

# Remember to commit the changes
conn.commit()

In [168]:
c.execute('SELECT * FROM houses').fetchall()

[(1, 4000, 5, 22, 619000)]

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [169]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
# 

c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

Select all rows from houses

In [171]:
c.execute('SELECT * FROM houses').fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000)]




#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [172]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('C:/Users/Monica/Desktop/IoD-Folders/MODULE 2/Labs///housing-data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [173]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [174]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [175]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In [176]:
c.execute('SELECT * FROM houses').fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000),
 (5, 2104, 3, 70, 399900),
 (6, 1600, 3, 28, 329900),
 (7, 2400, 3, 44, 369000),
 (8, 1416, 2, 49, 232000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (11, 1534, 3, 12, 314900),
 (12, 1427, 3, 57, 198999),
 (13, 1380, 3, 14, 212000),
 (14, 1494, 3, 15, 242500),
 (15, 1940, 4, 7, 239999),
 (16, 2000, 3, 27, 347000),
 (17, 1890, 3, 45, 329999),
 (18, 4478, 5, 49, 699900),
 (19, 1268, 3, 58, 259900),
 (20, 2300, 4, 77, 449900),
 (21, 1320, 2, 62, 299900),
 (22, 1236, 3, 78, 199900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (25, 1767, 3, 44, 252900),
 (26, 1888, 2, 79, 255000),
 (27, 1604, 3, 13, 242900),
 (28, 1962, 4, 53, 259900),
 (29, 3890, 3, 36, 573900),
 (30, 1100, 3, 60, 249900),
 (31, 1458, 3, 29, 464500),
 (32, 2526, 3, 13, 469000),
 (33, 2200, 3, 28, 475000),
 (34, 2637, 3, 25, 299900),
 (35, 1839, 2, 40, 349900),
 (36, 1000, 1, 5, 169900),
 (37

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [177]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

In [178]:
c.execute('SELECT * FROM houses').fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000),
 (5, 2104, 3, 70, 399900),
 (6, 1600, 3, 28, 329900),
 (7, 2400, 3, 44, 369000),
 (8, 1416, 2, 49, 232000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (11, 1534, 3, 12, 314900),
 (12, 1427, 3, 57, 198999),
 (13, 1380, 3, 14, 212000),
 (14, 1494, 3, 15, 242500),
 (15, 1940, 4, 7, 239999),
 (16, 2000, 3, 27, 347000),
 (17, 1890, 3, 45, 329999),
 (18, 4478, 5, 49, 699900),
 (19, 1268, 3, 58, 259900),
 (20, 2300, 4, 77, 449900),
 (21, 1320, 2, 62, 299900),
 (22, 1236, 3, 78, 199900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (25, 1767, 3, 44, 252900),
 (26, 1888, 2, 79, 255000),
 (27, 1604, 3, 13, 242900),
 (28, 1962, 4, 53, 259900),
 (29, 3890, 3, 36, 573900),
 (30, 1100, 3, 60, 249900),
 (31, 1458, 3, 29, 464500),
 (32, 2526, 3, 13, 469000),
 (33, 2200, 3, 28, 475000),
 (34, 2637, 3, 25, 299900),
 (35, 1839, 2, 40, 349900),
 (36, 1000, 1, 5, 169900),
 (37

In [179]:
data

[[None, 2104, 3, 70, 399900],
 [None, 1600, 3, 28, 329900],
 [None, 2400, 3, 44, 369000],
 [None, 1416, 2, 49, 232000],
 [None, 3000, 4, 75, 539900],
 [None, 1985, 4, 61, 299900],
 [None, 1534, 3, 12, 314900],
 [None, 1427, 3, 57, 198999],
 [None, 1380, 3, 14, 212000],
 [None, 1494, 3, 15, 242500],
 [None, 1940, 4, 7, 239999],
 [None, 2000, 3, 27, 347000],
 [None, 1890, 3, 45, 329999],
 [None, 4478, 5, 49, 699900],
 [None, 1268, 3, 58, 259900],
 [None, 2300, 4, 77, 449900],
 [None, 1320, 2, 62, 299900],
 [None, 1236, 3, 78, 199900],
 [None, 2609, 4, 5, 499998],
 [None, 3031, 4, 21, 599000],
 [None, 1767, 3, 44, 252900],
 [None, 1888, 2, 79, 255000],
 [None, 1604, 3, 13, 242900],
 [None, 1962, 4, 53, 259900],
 [None, 3890, 3, 36, 573900],
 [None, 1100, 3, 60, 249900],
 [None, 1458, 3, 29, 464500],
 [None, 2526, 3, 13, 469000],
 [None, 2200, 3, 28, 475000],
 [None, 2637, 3, 25, 299900],
 [None, 1839, 2, 40, 349900],
 [None, 1000, 1, 5, 169900],
 [None, 2040, 4, 75, 314900],
 [None, 3137,

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [180]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)') # WHERE: conditions. TRUNCATE is faster specially if you are going to delet a lot of data
conn.commit()

In [181]:
c.execute('SELECT * FROM houses').fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000),
 (5, 2104, 3, 70, 399900),
 (6, 1600, 3, 28, 329900),
 (7, 2400, 3, 44, 369000),
 (8, 1416, 2, 49, 232000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (11, 1534, 3, 12, 314900),
 (12, 1427, 3, 57, 198999),
 (13, 1380, 3, 14, 212000),
 (14, 1494, 3, 15, 242500),
 (15, 1940, 4, 7, 239999),
 (16, 2000, 3, 27, 347000),
 (17, 1890, 3, 45, 329999),
 (18, 4478, 5, 49, 699900),
 (19, 1268, 3, 58, 259900),
 (20, 2300, 4, 77, 449900),
 (21, 1320, 2, 62, 299900),
 (22, 1236, 3, 78, 199900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (25, 1767, 3, 44, 252900),
 (26, 1888, 2, 79, 255000),
 (27, 1604, 3, 13, 242900),
 (28, 1962, 4, 53, 259900),
 (29, 3890, 3, 36, 573900),
 (30, 1100, 3, 60, 249900),
 (31, 1458, 3, 29, 464500),
 (32, 2526, 3, 13, 469000),
 (33, 2200, 3, 28, 475000),
 (34, 2637, 3, 25, 299900),
 (35, 1839, 2, 40, 349900),
 (36, 1000, 1, 5, 169900),
 (37

QUIZ: What would `DELETE FROM houses` do?

In [ ]:
# Deleting the mentioned rows: 52 & 53

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [182]:
# ANSWER
results = c.execute("SELECT * FROM houses WHERE bdrms = 4")
results.fetchall()

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [183]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1827.5555555555557, 322892.51851851854),
 (4, 2399.733333333333, 373553.13333333336),
 (5, 4239.0, 659450.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [184]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [185]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('C:/Users/Monica/Desktop/IoD-Folders/MODULE 2/Labs///housing-data.csv', low_memory = False) # no failing in processing
data

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900
5,1985,4,61,299900
6,1534,3,12,314900
7,1427,3,57,198999
8,1380,3,14,212000
9,1494,3,15,242500


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [186]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False) 
# One table houses and second table housespandas. 
# con sql database in directories

Run a query to get the average price of each house size from this table:

In [187]:
#ANSWER
c.execute("SELECT bdrms, AVG(price) FROM houses_pandas GROUP BY bdrms").fetchall()

[(1, 169900.0),
 (2, 280866.6666666667),
 (3, 326403.92),
 (4, 377449.78571428574),
 (5, 699900.0)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

<a name="guided-practice"></a>


# Reference: SQL Syntax 


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [188]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [189]:
# Reading CSV to Dataframe
orders = pd.read_csv('C:/Users/Monica/Desktop/IoD-Folders/MODULE 2/Labs/P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('C:/Users/Monica/Desktop/IoD-Folders/MODULE 2/Labs/P12-OrderBreakdown.csv', encoding = 'utf-8')

In [190]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [191]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [192]:
# ANSWER
# orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

orders.columns = orders.columns.str.replace(' ', '_').str.lower()
orders

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
...,...,...,...,...
4112,ES-2014-2815584,2014-12-31,Erica Smith,Germany
4113,ES-2014-3458802,2014-12-31,John Grady,United Kingdom
4114,ES-2014-3638865,2014-12-31,Susan Vittorini,United Kingdom
4115,ES-2014-4785777,2014-12-31,Dennis Pardue,Germany


In [193]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes 

In [194]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [195]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [196]:
# Establishing Local DB connection
db_connection = sqlite3.connect('C:/Users/Monica/Desktop/IoD-Folders/MODULE 2/Labs/eshop.db.sqlite')

# Save these two dataframes as a table in sqlite
orders.to_sql(name = 'orders', con = db_connection, if_exists = 'replace', index = False)
orders_break_down.to_sql(name = 'orders_break_down', con = db_connection, if_exists = 'replace', index=False)

In [197]:
db_connection.execute('SELECT * FROM orders').fetchall()

[('IT-2011-3647632', '2011-01-01', 'Eugene Moren', 'Sweden'),
 ('ES-2011-4869686', '2011-01-03', 'Dorothy Dickinson', 'United Kingdom'),
 ('ES-2011-4939443', '2011-01-04', 'Arthur Prichep', 'France'),
 ('IT-2011-2942451', '2011-01-04', 'Grant Thornton', 'United Kingdom'),
 ('ES-2011-3848439', '2011-01-05', 'Michael Granlund', 'France'),
 ('ES-2011-5433855', '2011-01-07', 'Dave Poirier', 'France'),
 ('IT-2011-4546695', '2011-01-08', 'Darren Powers', 'France'),
 ('ES-2011-1138719', '2011-01-11', 'Eric Murdock', 'Italy'),
 ('ES-2011-1466305', '2011-01-11', 'Mick Brown', 'Austria'),
 ('ES-2011-4359424', '2011-01-11', 'Dorothy Dickinson', 'Spain'),
 ('ES-2011-5158390', '2011-01-11', 'Roy Collins', 'United Kingdom'),
 ('ES-2011-1460199', '2011-01-12', 'Roy Phan', 'Germany'),
 ('ES-2011-4976144', '2011-01-12', 'Anthony Garverick', 'United Kingdom'),
 ('ES-2011-5040255', '2011-01-12', 'Peter B?hler', 'United Kingdom'),
 ('IT-2011-5340302', '2011-01-13', 'Greg Matthias', 'Netherlands'),
 ('IT-2

#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [198]:
# When queries become so long, better to instantiate them as a string.
# Select first 10 rows of all columns in orders table

query = """
        SELECT * 
        FROM orders 
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

# Dataframe with some features

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [199]:
# Select Number of Orders for Each Customer
sql_string = """
    SELECT
        customer_name,
        COUNT(DISTINCT orders.order_id) AS Count
    FROM orders
    JOIN orders_break_down ON orders_break_down.order_id = orders.order_id
    GROUP BY customer_name ORDER BY Count DESC
"""

# Read a SQL query into pandas dataframe
customers = sql.read_sql(sql_string, con = db_connection)

customers.head()

,customer_name,Count
0,John Grady,13
1,Joel Jenkins,12
2,Aaron Smayling,12
3,Yoseph Carroll,11
4,Sarah Brown,11


In [200]:
db_connection.execute(sql_string).fetchall()

[('John Grady', 13),
 ('Joel Jenkins', 12),
 ('Aaron Smayling', 12),
 ('Yoseph Carroll', 11),
 ('Sarah Brown', 11),
 ("Patrick O'Brill", 11),
 ('Parhena Norris', 11),
 ('Monica Federle', 11),
 ('Kristen Hastings', 11),
 ('Jill Fjeld', 11),
 ('Frank Olsen', 11),
 ('Cynthia Delaney', 11),
 ('Art Foster', 11),
 ('Anthony Rawles', 11),
 ('Aaron Bergman', 11),
 ('Susan Pistek', 10),
 ('Shui Tom', 10),
 ('Sanjit Engle', 10),
 ("Patrick O'Donnell", 10),
 ('Natalie Fritzler', 10),
 ('Michelle Ellison', 10),
 ('Michael Grace', 10),
 ('Maria Etezadi', 10),
 ('Justin Hirsh', 10),
 ('Joy Bell-', 10),
 ('Jane Waco', 10),
 ('Gary Hansen', 10),
 ('Erica Smith', 10),
 ('Denny Joy', 10),
 ('Cari MacIntyre', 10),
 ('Bill Eplett', 10),
 ('Becky Castell', 10),
 ('Andrew Gjertsen', 10),
 ('Zuschuss Carroll', 9),
 ('Trudy Glocke', 9),
 ('Todd Sumrall', 9),
 ('Rob Dowd', 9),
 ('Phillip Flathmann', 9),
 ('Paul Stevenson', 9),
 ('Noel Staavos', 9),
 ('Michael Paige', 9),
 ('Michael Dominguez', 9),
 ('Maxwell S

#### 6. Select Number of Customers for Each Country

In [201]:
# Select Number of Customers for Each Country

sql_string = """
    SELECT
        country,
        COUNT(DISTINCT customer_name) AS Count
    FROM orders
    GROUP BY country
    ORDER BY Count DESC
"""

customers = sql.read_sql(sql_string, con = db_connection)

customers.head()

,country,Count
0,France,568
1,Germany,505
2,United Kingdom,473
3,Italy,351
4,Spain,306


In [ ]:
db_connection.execute(sql_string).fetchall()

#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [202]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table

sql_string = """
    SELECT
        discount
        , sales
        , quantity
    FROM orders_break_down
"""

customers = sql.read_sql(sql_string, con = db_connection)

customers.head()

,discount,sales,quantity
0,0.5,45.0,3
1,0.0,854.0,7
2,0.0,140.0,3
3,0.5,27.0,2
4,0.5,17.0,2


In [203]:
db_connection.execute(sql_string).fetchall()

[(0.5, 45.0, 3),
 (0.0, 854.0, 7),
 (0.0, 140.0, 3),
 (0.5, 27.0, 2),
 (0.5, 17.0, 2),
 (0.0, 90.0, 3),
 (0.0, 207.0, 4),
 (0.1, 155.0, 1),
 (0.0, 33.0, 3),
 (0.1, 716.0, 4),
 (0.6, 987.0, 6),
 (0.5, 116.0, 5),
 (0.0, 1384.0, 3),
 (0.0, 103.0, 2),
 (0.6, 268.0, 2),
 (0.0, 55.0, 1),
 (0.0, 97.0, 2),
 (0.0, 40.0, 5),
 (0.0, 22.0, 3),
 (0.5, 34.0, 2),
 (0.0, 290.0, 5),
 (0.1, 552.0, 5),
 (0.0, 257.0, 5),
 (0.0, 17.0, 2),
 (0.0, 522.0, 2),
 (0.5, 9.0, 3),
 (0.5, 22.0, 4),
 (0.5, 50.0, 2),
 (0.5, 198.0, 2),
 (0.5, 31.0, 5),
 (0.0, 224.0, 11),
 (0.0, 486.0, 3),
 (0.0, 440.0, 6),
 (0.1, 249.0, 1),
 (0.0, 21.0, 2),
 (0.0, 170.0, 5),
 (0.0, 55.0, 2),
 (0.0, 527.0, 2),
 (0.5, 17.0, 3),
 (0.0, 108.0, 8),
 (0.5, 26.0, 1),
 (0.0, 145.0, 3),
 (0.4, 48.0, 3),
 (0.0, 48.0, 3),
 (0.0, 58.0, 2),
 (0.0, 495.0, 2),
 (0.0, 825.0, 2),
 (0.0, 12.0, 2),
 (0.0, 666.0, 5),
 (0.0, 54.0, 1),
 (0.4, 889.0, 7),
 (0.6, 73.0, 1),
 (0.6, 69.0, 3),
 (0.0, 41.0, 4),
 (0.0, 54.0, 5),
 (0.0, 55.0, 3),
 (0.0, 28.0, 1),
 (0

#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [204]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table

sql_string = """
    SELECT
        discount
        , sales
        , quantity
        , (sales * quantity - discount) AS total_price
    FROM orders_break_down
    ORDER BY total_price DESC
"""

customers = sql.read_sql(sql_string, con = db_connection)

customers.head()

,discount,sales,quantity,total_price
0,0.1,5729.0,14,80205.9
1,0.0,6517.0,12,78204.0
2,0.1,4748.0,14,66471.9
3,0.1,5726.0,10,57259.9
4,0.0,4141.0,13,53833.0


In [205]:
db_connection.execute(sql_string).fetchall()

[(0.1, 5729.0, 14, 80205.9),
 (0.0, 6517.0, 12, 78204.0),
 (0.1, 4748.0, 14, 66471.9),
 (0.1, 5726.0, 10, 57259.9),
 (0.0, 4141.0, 13, 53833.0),
 (0.0, 5274.0, 10, 52740.0),
 (0.0, 5785.0, 9, 52065.0),
 (0.0, 5725.0, 9, 51525.0),
 (0.1, 4544.0, 11, 49983.9),
 (0.1, 5277.0, 9, 47492.9),
 (0.15, 4877.0, 9, 43892.85),
 (0.0, 3201.0, 13, 41613.0),
 (0.1, 4618.0, 9, 41561.9),
 (0.5, 3400.0, 12, 40799.5),
 (0.0, 2859.0, 14, 40026.0),
 (0.5, 2830.0, 13, 36789.5),
 (0.0, 3617.0, 10, 36170.0),
 (0.15, 4449.0, 8, 35591.85),
 (0.35, 3063.0, 11, 33692.65),
 (0.15, 2910.0, 11, 32009.85),
 (0.0, 4453.0, 7, 31171.0),
 (0.0, 3422.0, 9, 30798.0),
 (0.15, 2764.0, 11, 30403.85),
 (0.1, 3178.0, 9, 28601.9),
 (0.5, 2571.0, 11, 28280.5),
 (0.0, 3979.0, 7, 27853.0),
 (0.0, 3979.0, 7, 27853.0),
 (0.15, 2457.0, 11, 27026.85),
 (0.0, 3802.0, 7, 26614.0),
 (0.0, 2880.0, 9, 25920.0),
 (0.5, 2876.0, 9, 25883.5),
 (0.0, 2843.0, 9, 25587.0),
 (0.15, 2753.0, 9, 24776.85),
 (0.0, 2699.0, 9, 24291.0),
 (0.0, 2020.0, 12

#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [206]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100

sql_string = """
    SELECT
        *
        , (sales * quantity - discount) AS total_price
    FROM orders_break_down
    WHERE sales * quantity - discount > 100
    ORDER BY total_price ASC
"""

customers = sql.read_sql(sql_string, con = db_connection)

customers.head()

,order_id,product_name,discount,sales,quantity,category,total_price
0,IT-2011-2668777,"Binney & Smith Canvas, Blue",0.5,51.0,2,Office Supplies,101.5
1,IT-2011-1165265,"Cardinal Binding Machine, Durable",0.5,51.0,2,Office Supplies,101.5
2,ES-2012-1763830,"Elite Scissors, High Speed",0.5,34.0,3,Office Supplies,101.5
3,IT-2012-4099397,"Sanford Canvas, Blue",0.5,51.0,2,Office Supplies,101.5
4,ES-2013-4250046,"Cardinal Binding Machine, Durable",0.5,51.0,2,Office Supplies,101.5


In [207]:
db_connection.execute(sql_string).fetchall()

[('IT-2011-2668777',
  'Binney & Smith Canvas, Blue',
  0.5,
  51.0,
  2,
  'Office Supplies',
  101.5),
 ('IT-2011-1165265',
  'Cardinal Binding Machine, Durable',
  0.5,
  51.0,
  2,
  'Office Supplies',
  101.5),
 ('ES-2012-1763830',
  'Elite Scissors, High Speed',
  0.5,
  34.0,
  3,
  'Office Supplies',
  101.5),
 ('IT-2012-4099397',
  'Sanford Canvas, Blue',
  0.5,
  51.0,
  2,
  'Office Supplies',
  101.5),
 ('ES-2013-4250046',
  'Cardinal Binding Machine, Durable',
  0.5,
  51.0,
  2,
  'Office Supplies',
  101.5),
 ('ES-2014-5784412',
  'Okidata Calculator, White',
  0.5,
  51.0,
  2,
  'Technology',
  101.5),
 ('IT-2014-5393609',
  'Tenex Lockers, Single Width',
  0.5,
  102.0,
  1,
  'Office Supplies',
  101.5),
 ('ES-2012-5499117',
  'Fellowes Box, Wire Frame',
  0.4,
  34.0,
  3,
  'Office Supplies',
  101.6),
 ('ES-2014-3096832',
  'Fellowes Box, Wire Frame',
  0.4,
  34.0,
  3,
  'Office Supplies',
  101.6),
 ('ES-2011-4665208',
  'Avery Binder Covers, Durable',
  0.1,
 

#### 8. Select All Customer And The Product They Have Bought

In [208]:
# Select All Customer And The Product They Have Bought

sql_string = """
    SELECT
        orders.order_id
        , orders.customer_name
        , orders_break_down.product_name
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
"""

customers = sql.read_sql(sql_string, con = db_connection)

customers.head()

,order_id,customer_name,product_name
0,IT-2011-3647632,Eugene Moren,"Enermax Note Cards, Premium"
1,ES-2011-4869686,Dorothy Dickinson,"Dania Corner Shelving, Traditional"
2,ES-2011-4939443,Arthur Prichep,"Binney & Smith Sketch Pad, Easy-Erase"
3,IT-2011-2942451,Grant Thornton,"Boston Markers, Easy-Erase"
4,IT-2011-2942451,Grant Thornton,"Eldon Folders, Single Width"


In [209]:
db_connection.execute(sql_string).fetchall()

[('IT-2011-3647632', 'Eugene Moren', 'Enermax Note Cards, Premium'),
 ('ES-2011-4869686',
  'Dorothy Dickinson',
  'Dania Corner Shelving, Traditional'),
 ('ES-2011-4939443',
  'Arthur Prichep',
  'Binney & Smith Sketch Pad, Easy-Erase'),
 ('IT-2011-2942451', 'Grant Thornton', 'Boston Markers, Easy-Erase'),
 ('IT-2011-2942451', 'Grant Thornton', 'Eldon Folders, Single Width'),
 ('ES-2011-3848439',
  'Michael Granlund',
  'Binney & Smith Pencil Sharpener, Water Color'),
 ('ES-2011-3848439', 'Michael Granlund', 'Sanford Canvas, Fluorescent'),
 ('ES-2011-5433855', 'Dave Poirier', 'Accos Thumb Tacks, Assorted Sizes'),
 ('ES-2011-5433855', 'Dave Poirier', 'Bush Floating Shelf Set, Pine'),
 ('ES-2011-5433855', 'Dave Poirier', 'Smead Lockers, Industrial'),
 ('IT-2011-4546695', 'Darren Powers', 'Binney & Smith Sketch Pad, Blue'),
 ('IT-2011-4546695', 'Darren Powers', 'Ikea Classic Bookcase, Metal'),
 ('ES-2011-1138719', 'Eric Murdock', 'Ibico Hole Reinforcements, Recycled'),
 ('ES-2011-1466305

#### 9.A Select Number of 'Furniture' Orders For Each Country 

In [210]:
# Select Number of 'Furniture' Orders For Each Country

sql_string = """
    SELECT
       orders.country
       , COUNT(orders_break_down.category) AS Count
    FROM orders
    JOIN orders_break_down
    ON orders.order_id = orders_break_down.order_id
    
    WHERE orders_break_down.category = 'Furniture'
    GROUP BY orders.country
"""

customers = sql.read_sql(sql_string, con = db_connection)

customers.head()

,country,Count
0,Austria,40
1,Belgium,20
2,Denmark,9
3,Finland,16
4,France,299


In [211]:
db_connection.execute(sql_string).fetchall()

[('Austria', 40),
 ('Belgium', 20),
 ('Denmark', 9),
 ('Finland', 16),
 ('France', 299),
 ('Germany', 264),
 ('Ireland', 12),
 ('Italy', 136),
 ('Netherlands', 76),
 ('Norway', 6),
 ('Portugal', 10),
 ('Spain', 123),
 ('Sweden', 36),
 ('Switzerland', 11),
 ('United Kingdom', 180)]

#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [212]:
# Select Number of 'Furniture' Orders For The Country Denmark

sql_string = """
    SELECT
       *
    FROM orders
    JOIN orders_break_down
    ON orders.order_id = orders_break_down.order_id
    WHERE orders_break_down.category = 'Furniture'
        AND orders.country = 'Denmark'
 """

customers = sql.read_sql(sql_string, con = db_connection)

customers.head()

,order_id,order_date,customer_name,country,order_id,product_name,discount,sales,quantity,category
0,ES-2011-1586186,2011-06-16,Vivek Grady,Denmark,ES-2011-1586186,"Office Star Bag Chairs, Set of Two",0.5,87.0,3,Furniture
1,IT-2012-4084224,2012-02-15,Mick Brown,Denmark,IT-2012-4084224,"Rubbermaid Door Stop, Ergonomic",0.6,17.0,1,Furniture
2,IT-2013-4649004,2013-05-31,Bart Folk,Denmark,IT-2013-4649004,"Novimex Executive Leather Armchair, Red",0.5,920.0,4,Furniture
3,ES-2013-3422294,2013-09-07,Dave Brooks,Denmark,ES-2013-3422294,"Advantus Door Stop, Duo Pack",0.6,36.0,2,Furniture
4,ES-2014-3653593,2014-02-10,Ivan Liston,Denmark,ES-2014-3653593,"Deflect-O Clock, Black",0.6,161.0,8,Furniture


In [213]:
db_connection.execute(sql_string).fetchall()

[('ES-2011-1586186',
  '2011-06-16',
  'Vivek Grady',
  'Denmark',
  'ES-2011-1586186',
  'Office Star Bag Chairs, Set of Two',
  0.5,
  87.0,
  3,
  'Furniture'),
 ('IT-2012-4084224',
  '2012-02-15',
  'Mick Brown',
  'Denmark',
  'IT-2012-4084224',
  'Rubbermaid Door Stop, Ergonomic',
  0.6,
  17.0,
  1,
  'Furniture'),
 ('IT-2013-4649004',
  '2013-05-31',
  'Bart Folk',
  'Denmark',
  'IT-2013-4649004',
  'Novimex Executive Leather Armchair, Red',
  0.5,
  920.0,
  4,
  'Furniture'),
 ('ES-2013-3422294',
  '2013-09-07',
  'Dave Brooks',
  'Denmark',
  'ES-2013-3422294',
  'Advantus Door Stop, Duo Pack',
  0.6,
  36.0,
  2,
  'Furniture'),
 ('ES-2014-3653593',
  '2014-02-10',
  'Ivan Liston',
  'Denmark',
  'ES-2014-3653593',
  'Deflect-O Clock, Black',
  0.6,
  161.0,
  8,
  'Furniture'),
 ('ES-2014-3653593',
  '2014-02-10',
  'Ivan Liston',
  'Denmark',
  'ES-2014-3653593',
  'Harbour Creations Bag Chairs, Set of Two',
  0.5,
  32.0,
  1,
  'Furniture'),
 ('ES-2014-1415600',
  '201

#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [214]:
# Select Total Sales With Discount and Without Discount for Each Country

sql_string = """
    SELECT
        orders.country
        , SUM(CASE WHEN discount == 0 THEN sales ELSE 0 END) AS discount_sales
        , SUM(CASE WHEN discount > 0 THEN sales ELSE 0 END) AS non_discount_sales
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country
 """

customers = sql.read_sql(sql_string, con = db_connection)

customers.head()

,country,discount_sales,non_discount_sales
0,Austria,79382.0,0.0
1,Belgium,42320.0,0.0
2,Denmark,0.0,7763.0
3,Finland,20702.0,0.0
4,France,129791.0,479892.0


#### 11.A Select Total Quantity, Total Sales for Each Country

In [215]:
# Select Total Quantity, Total Sales for Each Country

sql_string = """
    SELECT
        orders.country
        , SUM(quantity) AS total_quantity
        , SUM(sales) AS total_sales
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country
 """

customers = sql.read_sql(sql_string, con = db_connection)

customers.head()

,country,total_quantity,total_sales
0,Austria,973,79382.0
1,Belgium,532,42320.0
2,Denmark,204,7763.0
3,Finland,201,20702.0
4,France,7329,609683.0


#### 11.B Select Top 3 Country Based on Sales

In [218]:
# Select Top 3 Country Based on Sales

sql_string = """
    SELECT
        orders.country
        , SUM(sales) AS total_sales 
        , SUM(quantity) AS total_quantity
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country
    ORDER BY total_sales DESC LIMIT 3
 """

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

,country,total_sales,total_quantity
0,France,609683.0,7329
1,Germany,488681.0,6179
2,United Kingdom,420497.0,4917


In [220]:
order_list.sort_values(by='total_sales', ascending=False).head(5)

,country,total_sales,total_quantity
0,France,609683.0,7329
1,Germany,488681.0,6179
2,United Kingdom,420497.0,4917


#### 11.C Select Bottom 3 Country Based On Quantities

In [221]:
# Select Bottom 3 Country Based On Quantities
# statements start with CASE and end with END (like if statements in Python)

sql_string = """
    SELECT
        orders.country
        , SUM(sales) AS total_sales
        , SUM(quantity) AS total_quantity
        
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country
    ORDER BY total_quantity ASC LIMIT 3
 """

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

,country,total_sales,total_quantity
0,Finland,20702.0,201
1,Denmark,7763.0,204
2,Norway,20529.0,261


In [222]:
order_list.sort_values(by='total_quantity', ascending=True).head(5)

,country,total_sales,total_quantity
0,Finland,20702.0,201
1,Denmark,7763.0,204
2,Norway,20529.0,261


#### 12. Select Average Sales By Categroy For The Country 'France'

In [223]:
# Select Average Sales By Categroy For The Country 'France'

sql_string = """
    SELECT
        orders.country
        , orders_break_down.category
        , AVG(sales) AS avg_sales
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
    WHERE orders.country = 'France'
    GROUP BY orders.country, orders_break_down.category
    ORDER BY avg_sales DESC
 """

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

,country,category,avg_sales
0,France,Technology,595.145078
1,France,Furniture,582.314381
2,France,Office Supplies,167.217709


#### 13. Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [224]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest

sql_string = """
    SELECT
        orders.country
        , orders_break_down.category
        , AVG(sales) AS total_sales
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country, orders_break_down.category
    ORDER BY total_sales DESC LIMIT 1
 """

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

,country,category,total_sales
0,Switzerland,Technology,902.846154


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax



---



---



> > > > > > > > > © 2019 Institute of Data


---



---



